In [1]:
from pydantic import BaseModel, Field
from collections.abc import Callable
from typing import List, Dict, Union, Optional, Any, ForwardRef, Literal
import os
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionMessage
from openai.types.chat.chat_completion_message_tool_call import ChatCompletionMessageToolCall
import logging
from collections import defaultdict
import pandas as pd
from tqdm.auto import tqdm
import json

load_dotenv()

logging.getLogger("httpx").setLevel(logging.WARNING)
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s - %(levelname)s - %(message)s")

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=OPENAI_API_KEY)


In [2]:
import os
os.chdir(f"{os.path.dirname(os.path.abspath(os.getcwd()))}/zavmo")

In [3]:
os.getcwd()

'c:\\Users\\smrit\\Work\\Kenpath\\zavmo-api\\zavmo'

In [4]:
from helpers.chat import filter_history
from pydantic import BaseModel
import logging
from helpers._types import (
    Agent,
    function_to_json,
    Result,
    Response, 
    StrictTool,
    PermissiveTool
)
from typing import List, Dict, Any
from collections import defaultdict
# from helpers.utils import get_utc_timestamp

In [5]:
sys="""
Based on the unit text provided, generate a comprehensive mark scheme and corresponding learning materials using Bloom's Taxonomy. 
Follow the defintion for every item in the mark scheme for each level of Bloom's Taxonomy and gain clarity on the criteria, task and response generation:

1: Develop Bloom's Taxonomy Criteria
	1.	For Each Level of Bloom's Taxonomy (Remembering, Understanding, Applying, Analyzing, Evaluating, and Creating):
	•	Define clear criteria for evaluating learners' work at each level.
	•	Use key verbs (e.g., "list," "explain," "demonstrate," "compare," "evaluate," "design") to frame expectations.
	•	Clearly articulate how learners can achieve each level, aligning criteria to the text provided.

2: Design Tasks for Learners
	1.	Create one task for each level of Bloom's Taxonomy that reflects the criteria defined in Step 1.
	•	Ensure the tasks are specific, measurable, and tailored to the unit text.
	•	Use varied formats (e.g., multiple-choice questions for Remembering, essays for Evaluating, or project-based tasks for Creating).

3: Generate Sample Responses with Grading Standards
	1.	Provide responses for each task which should set OFQUAL standards.
	2.	Align responses to the following performance bands with each item having this structure:
        level: for which the response scheme is set.
        response: variation of responses that should set OFQUAL standards.
		•	Fail (39% or less): Responses demonstrating minimal understanding or effort.
		•	Pass (40%-59%): Responses meeting basic expectations but with limited depth.
		•	Merit (60%-79%): Responses showing solid understanding, detail, and application.
		•	Distinction (80%-100%): Responses exemplifying comprehensive understanding, insight, and creativity.
"""

# agent = Agent(
#     name="Generate Mark Scheme",
#     functions=[GenerateCriteriaOnAllLevels, GenerateBenchmarkingResponseOnAllLevels],
#     model="gpt-4o-mini",
#     instructions=sys
# )

In [6]:
system_message = {'role':'system','content':sys}
ofqual_unitwise_text = """UNIT 1:
- Unit Title/Heading: Knowledge of the Service and Repair of Electrically Propelled Buses and Coaches
- Unit Aim/Purpose: This unit is for people who work on or near electric and electric/hybrid buses and coaches. It includes essential knowledge of the hazards associated with electric and electric/hybrid buses and coaches and the precautions to follow to avoid these. It enables the learner to understand how to safely isolate and reinstate the high voltage system and to remove and replace high voltage components.
- Level: 2
- Reference Number: L/618/5652
- Credit Value: 2
- Total Qualification Time (TQT): 19
- Guided Learning Hours: 16
- Learning Outcomes:
  1. Understand the risks and hazards associated with electric and electric/hybrid buses and coaches
  2. Know and understand the different types of electric and electric/hybrid buses and coaches, associated technology, components and operating principles
  3. Know how to prepare electric and electric/hybrid buses and coaches when carrying out routine service and repair procedures
  4. Know how to work safely on electric and electric/hybrid buses and coaches
- Assessment Criteria:
  1.1 Describe the health and safety legislation and workplace procedures relating to working on, near or with electric and electric/hybrid buses and coaches.
  1.2 Describe the dangers relating to working with high voltages, electrocution, battery electrolyte gel, and hazards associated with alternative fuel sources and systems, including hydrogen fuel cells.
  1.3 Describe safety requirements including First Aid, tools and equipment, ventilation, high voltage isolation, dealing with electrolyte gel spillages, environmental protection, risk assessment, and workplace signage.
  1.4 Describe vehicle power systems and their associated safety risks.
  2.1 Identify components that make up the electric and electric/hybrid systems.
  2.2 Describe basic operating principles.
  2.3 Describe the construction and function of battery types.
  2.4 Describe the construction and function of component parts.
  2.5 Describe how to store parts and components.
  3.1 Describe the preparation of the vehicle prior to conducting service/repair of the vehicle.
  4.1 Describe safe working methods for working on electric and electric/hybrid buses and coaches.

"""

## Finalized approach for generating markscheme

In [8]:
class GenerateCriteria(StrictTool):
    blooms_taxonomy_level: Literal["Remember", "Understand", "Apply", "Analyze", "Evaluate", "Create"] = Field(description="The level of Bloom's Taxonomy.")
    criteria: List[str] = Field(description="Generate very challenging criteria for the level of Bloom's Taxonomy based on the ofqual unit provided.")
    key_verbs: List[str] = Field(description="Action verbs that align with Bloom's Taxonomy level to specify the expected learning outcomes (e.g., 'define,' 'explain,' 'apply,' 'analyze,' 'evaluate,' 'create').")
    expectations: List[str] = Field(description="Expectations to meet the criteria.")
    task: str = Field(description="A task designed around the level of Bloom's Taxonomy based on the ofqual unit and criteria.")

class GenerateCriteriaOnAllLevels(StrictTool):
    task_and_criteria: List[GenerateCriteria] = Field(description="Generate criteria for all levels of Bloom's Taxonomy based on the ofqual unit provided.")
    def execute(self, context: Dict):
        return Result(value="Tasks and corresponding criteria on all levels of Bloom's Taxonomy: \n\n" + "\n".join([f"{item.task}:\n\n{item.criteria}" for item in self.task_and_criteria])+"\n\nNext step is to generate a benchmarking response for each task.")

class pass_fail_merit_distinction(StrictTool):
    """
    This class is used to generate benchmarking responses for the pass, fail, merit, and distinction for the markscheme item.
    """
    fail: str = Field(description="Response demonstrating minimal understanding or effort, showing insufficient grasp of core concepts")
    pass_: str = Field(description="Response meeting basic expectations with limited depth, showing fundamental understanding of core concepts")
    merit: str = Field(description="Response showing solid understanding, detail, and application, demonstrating thorough comprehension and ability to apply concepts")
    distinction: str = Field(description="Response exemplifying comprehensive understanding, insight, and creativity, showing exceptional mastery and innovative application of concepts")

class GenerateBenchmarkingResponse(StrictTool):
    response: List[pass_fail_merit_distinction] = Field(description="Generate benchmarking responses for the task to have variations in response considering criteria and expectations provided.")
    def execute(self, context: Dict):
        return Result(value=self.response)

def get_structured_output(func, messages=[],model="gpt-4o-mini"):
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    completion = client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    response_format=func,
    )
    
    return completion.choices[0].message.parsed    
    
def generate_markscheme(unit_text: str):
    messages = [system_message,
    {"role": "user", "content": f"Here is the Ofqual(Office of Qualifications and Examinations Regulation) unitwise text: {unit_text}"}]
    criteria_items = get_structured_output(GenerateCriteriaOnAllLevels, messages)

    marks_scheme = []
    for i in criteria_items.task_and_criteria:
        current_item = {}
        current_item['bloom_taxonomy_level'] = i.blooms_taxonomy_level
        current_item['criteria'] = i.criteria
        current_item['key_verbs'] = i.key_verbs
        current_item['expectations'] = i.expectations
        current_item['task'] = i.task
        
        messages = [system_message,
        {"role": "user", "content": f"Generate benchmarking responses for the task based on OFQUAL standards: {i.task},\n\nCriteria: {i.criteria},\n\nExpectations: {i.expectations}"}]
        responses = [i for i in get_structured_output(GenerateBenchmarkingResponse, messages).response[0]]
        current_item['benchmarking_responses'] = responses
        marks_scheme.append(current_item)
        
    return marks_scheme


In [15]:
df = pd.read_csv(r"C:\Users\smrit\Work\Kenpath\zavmo-api\docs\rgcn\ofqual_units.csv")

In [16]:
df.columns

Index(['ofqual_id', 'overview', 'unit_id', 'unit_title', 'unit_description',
       'unit_learning_outcomes', 'qualification_type', 'qualification_level',
       'assessment_methods', 'sector_subject_area', 'awarding_organisation',
       'total_credits', 'guided_learning_hours', 'total_qualification_time',
       'awarding_organization'],
      dtype='object')

In [17]:
df.drop_duplicates(subset='unit_id', inplace=True)

In [26]:
df.shape

(14343, 15)

In [28]:
df['text'] = df.apply(lambda row: f"Unit Title: {row['unit_title']}\n  - Unit Aim: {row['unit_description']}\n  - Unit id: {row['unit_id']}\n  - Credit Value: {row['total_credits']}\n  - Total Qualification Time (TQT): {row['total_qualification_time']}\n  - Guided Learning Hours (GLH): {row['guided_learning_hours']}\n  - Learning Outcomes:\n    {row['unit_learning_outcomes']}", axis=1)

In [29]:
df['text'][0]

'Unit Title: Alcohol and Drug Misuse Awareness\n  - Unit Aim: This unit gives learners an understanding of the effects and consequences of drug and alcohol misuse.\n  - Unit id: T/504/8484\n  - Credit Value: 3.0\n  - Total Qualification Time (TQT): 30.0\n  - Guided Learning Hours (GLH): 15.0\n  - Learning Outcomes:\n    1. Understand the effects and consequences of alcohol misuse.\r\n  - Describe the physical effects of alcohol misuse\r\n  - Identify social consequences of alcohol misuse'

In [30]:
df.to_csv(r"C:\Users\smrit\Work\Kenpath\zavmo-api\docs\rgcn\ofqual_units_with_text.csv", index=False)

In [ ]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

def process_row(row):
    """Process a single row and return (index, markscheme)"""
    try:
        markscheme = generate_markscheme(row['text'])
        return row.name, markscheme
    except Exception as e:
        print(f"Error processing row {row.name}: {str(e)}")
        return row.name, None

def process_with_threadpool(df, max_workers=8, batch_size=2000):
    """Process dataframe using ThreadPoolExecutor with batching"""
    all_markschemes = {}
    total_batches = (len(df) + batch_size - 1) // batch_size
    
    with tqdm(total=len(df), desc="Generating markschemes") as pbar:
        for batch_start in range(0, len(df), batch_size):
            batch_end = min(batch_start + batch_size, len(df))
            batch_df = df.iloc[batch_start:batch_end]
            
            # Process batch with ThreadPoolExecutor
            with ThreadPoolExecutor(max_workers=max_workers) as executor:
                # Submit all rows in the batch
                future_to_row = {
                    executor.submit(process_row, row): idx 
                    for idx, row in batch_df.iterrows()
                }
                
                # Collect results as they complete
                for future in as_completed(future_to_row):
                    try:
                        idx, markscheme = future.result()
                        all_markschemes[idx] = markscheme
                        pbar.update(1)
                    except Exception as e:
                        print(f"Error processing future: {str(e)}")
            
            # Optional: Save intermediate results after each batch
            save_intermediate_results(df, all_markschemes, batch_start//batch_size)
            
    
    return all_markschemes

def save_intermediate_results(df, markschemes, batch_num):
    """Save intermediate results after each batch"""
    try:
        temp_df = df.copy()
        temp_df['markscheme'] = pd.Series(markschemes)
        temp_df.to_csv(f'intermediate_markschemes_batch_{batch_num}.csv', index=False)
    except Exception as e:
        print(f"Error saving intermediate results: {str(e)}")

# Main execution
if __name__ == "__main__":
    # Read the CSV file
    df = pd.read_csv(r"C:\Users\smrit\Work\Kenpath\zavmo-api\docs\rgcn\ofqual_units_with_text.csv").head(10000)
    
    # Process with ThreadPool
    markschemes_dict = process_with_threadpool(
        df,
        max_workers=8,  # Adjust based on your CPU and API limits
        batch_size=2000  # Adjust based on your memory and requirements
    )
    
    # Convert results to series and add to dataframe
    df['markscheme'] = pd.Series(markschemes_dict)
    
    # Save the final results
    df.to_csv(r"C:\Users\smrit\Work\Kenpath\zavmo-api\docs\rgcn\ofqual_units_with_markscheme.csv", index=False)

Generating markschemes:   1%|          | 109/10000 [10:15<19:05:50,  6.95s/it]

Error processing row 111: Could not parse response content as the length limit was reached - CompletionUsage(completion_tokens=59, prompt_tokens=751, total_tokens=810, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))


Generating markschemes:   4%|▍         | 445/10000 [41:35<11:17:51,  4.26s/it]

Error processing row 449: Could not parse response content as the length limit was reached - CompletionUsage(completion_tokens=54, prompt_tokens=749, total_tokens=803, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))


Generating markschemes:   5%|▌         | 504/10000 [46:44<9:42:03,  3.68s/it] 

...

Generating markschemes:  40%|███▉      | 3996/10000 [11:26:36<16:18:49,  9.78s/it]

Error processing row 3995: Connection error.
Error processing row 3996: Connection error.
Error processing row 3997: Connection error.


Generating markschemes:  40%|███▉      | 3998/10000 [11:26:36<11:13:23,  6.73s/it]

Error processing row 3998: Connection error.
Error processing row 3999: Connection error.


Generating markschemes:  40%|███▉      | 3999/10000 [11:26:53<11:13:17,  6.73s/it]2025-03-19 11:09:35,710 - INFO - Retrying request to /chat/completions in 0.409050 seconds
2025-03-19 11:09:36,126 - INFO - Retrying request to /chat/completions in 0.937029 seconds
Generating markschemes:  40%|████      | 4008/10000 [11:28:12<10:04:58,  6.06s/it]

In [ ]:
df = pd.read_csv(r"C:\Users\smrit\Work\Kenpath\zavmo-api\docs\rgcn\ofqual_units_with_text.csv").head(5000)

# Process the dataset with tqdm
markschemes = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Generating markschemes"):
    markscheme = generate_markscheme(row['text'])
    print(markscheme)
    markschemes.append(markscheme)

df['markscheme'] = markschemes

# Save the updated dataset
df.to_csv(r"C:\Users\smrit\Work\Kenpath\zavmo-api\docs\rgcn\ofqual_units_with_markscheme.csv", index=False)

In [89]:
markschemes[1]

[{'bloom_taxonomy_level': 'Remember',
  'criteria': ['Recall key information related to health and safety legislation and workplace procedures.',
   'Identify different types of electric and electric/hybrid buses and coaches.'],
  'key_verbs': ['list', 'identify', 'recall'],
  'expectations': ['Learners should provide a list of relevant legislation and procedures without interpretation.',
   'Learners should accurately enumerate types without explaining their differences.'],
  'task': 'List the health and safety legislation and workplace procedures relating to working on electric and electric/hybrid buses and coaches.',
  'benchmarking_responses': [('fail',
    "The response lacks detail and may only list one or two pieces of legislation, such as 'Health and Safety at Work Act' without further context or additional workplace procedures."),
   ('pass_',
    "The response includes a basic list of several pieces of health and safety legislation such as 'Health and Safety at Work Act', 'Ma

In [13]:
import pandas as pd
df = pd.read_excel(r"C:\Users\smrit\Downloads\NOS & OFQUAL\ofqual_energy_units_with_markscheme.xlsx")

